In [1]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
from cellpose.io import imread
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def get_measurements(mask,props):
    df = sk.measure.regionprops_table(mask,img,properties=props)
    return df

In [ ]:
def save(save_path, img_name, masks, flows, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        flows_path = os.mkdir(os.path.join(save_path,'flows'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        flows_path = os.path.join(save_path,'flows')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'masks_'+img_name[:-4]+'.tif'),mask_img)
    imsave(os.path.join(selected_spots_path,'flows_'+img_name[:-4]+'.tif'),selected_spots)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

In [ ]:
all_files = sorted(glob('E:/Lien_Lab/Sam/OIC-130_CellCountScript/All_Imgs/*.tif'))
all_img = list(map(sk.io.imread,all_files))

In [ ]:
io.logger_setup()
model = models.CellposeModel(gpu=True,pretrained_model='IncucyteV4')
#model = models.Cellpose(gpu=True,pretrained_model='E:/Lien_Lab/Sam/OIC-130_CellCountScript/All_Imgs/models/IncucyteV4') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]

In [ ]:
props = ['label','area']
save_path = 'E:/Lien_Lab/Sam/OIC-130_CellCountScript/'
for i in range(len(masks)):
    img_name = os.path.basename(all_files[i])
    masks, flows, styles, diams = model.eval(all_img[i], diameter=20, channels=channels,flow_threshold=0.4)
    df = get_measurements(masks,props)
    save(save_path, img_name, masks, flows, merged_df=df)